In [2]:
import json
from tqdm import tqdm
from together import Together
from utils import *

In [3]:
def add_world_model_probs_single(value, token_label, prompt, client, world_model_endpoints, is_star=True):
    data_key = 'y_stars' if is_star else 'y_NON_stars'
    
    if 'ents_prob' in value[data_key][token_label]:
        remaining_ents = list(value[data_key][token_label]['ents_prob'].keys())
    else:
        remaining_ents = []

    if 'world_models' not in value[data_key][token_label]:
        value[data_key][token_label]['world_models'] = []
    if 'world_ents_prob' not in value[data_key][token_label]:
        value[data_key][token_label]['world_ents_prob'] = []
        
    #for wm_endpoint in tqdm(world_model_endpoints, desc="world model endpoints", leave=False):
    for wm_endpoint in world_model_endpoints:
        wm_prob_gen = GenerateNextTokenProbAPI(client, wm_endpoint)
        max_tokens=len(wm_prob_gen.tokenizer(prompt)['input_ids'])+10
        # wm_prob = compute_token_probs_api(token_label, prompt, wm_prob_gen, max_tokens)
        wm_result = wm_prob_gen.get_token_probs(
            prompt=prompt,
            target_string=token_label,
            remaining_ents=remaining_ents,
            max_tokens=max_tokens
        )

        if wm_result == -1:
            # if the target span wasn't found
            target_prob = -1.0
            ents_prob = {}
        else:
            target_prob = float(wm_result["target_prob"])
            ents_prob = {ent: float(prob) for ent, prob in wm_result["ents_prob"].items()}

        # value[data_key][token_label]['world_models'].append(float(wm_prob))
        value[data_key][token_label]['world_models'].append(target_prob)
        value[data_key][token_label]['world_ents_prob'].append(ents_prob)

def add_shadow_model_probs_single(value, token_label, prompt, client, shadow_model_endpoints, is_star=True):
    data_key = 'y_stars' if is_star else 'y_NON_stars'

    if 'ents_prob' in value[data_key][token_label]:
        remaining_ents = list(value[data_key][token_label]['ents_prob'].keys())
    else:
        remaining_ents = []
    
    if 'shadow_models' not in value[data_key][token_label]:
        value[data_key][token_label]['shadow_models'] = []
    if 'shadow_ents_prob' not in value[data_key][token_label]:
        value[data_key][token_label]['shadow_ents_prob'] = []
        
    #for sm_endpoint in tqdm(shadow_model_endpoints, desc="shadow model endpoints", leave=False):
    for sm_endpoint in shadow_model_endpoints:
        sm_prob_gen = GenerateNextTokenProbAPI(client, sm_endpoint)
        max_tokens=len(sm_prob_gen.tokenizer(prompt)['input_ids'])+10
        # sm_prob = compute_token_probs_api(token_label, prompt, sm_prob_gen, max_tokens)

        sm_result = sm_prob_gen.get_token_probs(
            prompt=prompt,
            target_string=token_label,
            remaining_ents=remaining_ents,
            max_tokens=max_tokens
        )

        if sm_result == -1:
            target_prob = -1.0
            ents_prob = {}
        else:
            target_prob = float(sm_result["target_prob"])
            ents_prob = {ent: float(prob) for ent, prob in sm_result["ents_prob"].items()}

        # value[data_key][token_label]['shadow_models'].append(float(sm_prob))
        value[data_key][token_label]['shadow_models'].append(target_prob)
        value[data_key][token_label]['shadow_ents_prob'].append(ents_prob)


def add_model_probs(results, client, world_model_endpoints, shadow_model_endpoints, model_type='world'):
    for t_id, value in tqdm(results.items(), desc="processing results"):
        print(f"processing {t_id}")
        y_stars_order = list(value['y_stars'].keys())
        y_non_stars_order = list(value['y_NON_stars'].keys())

        # y_stars 
        for y_star in tqdm(y_stars_order, leave=True, desc="processing y_stars"):
            prompt = value['y_stars'][y_star]['prompt']
            if model_type == 'world':
                add_world_model_probs_single(value, y_star, prompt, client, world_model_endpoints, is_star=True)
            else:
                add_shadow_model_probs_single(value, y_star, prompt, client, shadow_model_endpoints, is_star=True)

        # y_non_stars 
        for y_non_star in tqdm(y_non_stars_order, leave=True, desc="processing y_non_stars"):
            prompt = value['y_NON_stars'][y_non_star]['prompt']
            if model_type == 'world':
                add_world_model_probs_single(value, y_non_star, prompt, client, world_model_endpoints, is_star=False)
            else:
                add_shadow_model_probs_single(value, y_non_star, prompt, client, shadow_model_endpoints, is_star=False)

In [4]:
id=3
with open('model_map.json') as f:
    model_map=json.load(f)
shadow_model_endpoints=[model_map[f'shadow_train_{id}']['api_key']]
shadow_model_endpoints

['bh193/Meta-Llama-3.1-8B-Instruct-Reference-507a0658-fa62ab3e']

In [5]:
with open(f'target_token_probs_train_{id}_10_epochs_prompt_2.json', 'r') as f:
    all_model_probs = json.load(f)    
key = '779d92de61a5035835e5023ca79e2e5b6124c6300c3ceb0e07e374f948554116'
client = Together(api_key=key)
world_model_endpoints = [
    "google/gemma-2b-it",
    "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    'mistralai/Mistral-7B-Instruct-v0.2'
]

In [6]:
add_model_probs(all_model_probs, client, [], shadow_model_endpoints, model_type='shadow')

processing results:   0%|          | 0/100 [00:00<?, ?it/s]

processing train_1


processing results:   1%|          | 1/100 [00:15<25:29, 15.45s/it]

processing train_17


processing results:   2%|▏         | 2/100 [00:27<21:53, 13.40s/it]

...

processing results:  98%|█████████▊| 98/100 [25:07<00:36, 18.14s/it]

processing test_644


processing results:  99%|█████████▉| 99/100 [25:17<00:15, 15.57s/it]

processing test_652


processing results: 100%|██████████| 100/100 [25:29<00:00, 15.29s/it]


In [7]:
with open(f'target_token_probs_train_{id}_10_epochs_prompt_2_with_shadow.json', 'w') as f:
    json.dump(all_model_probs, f)

In [8]:
add_model_probs(all_model_probs, client, world_model_endpoints, [], model_type='world')

processing results:   0%|          | 0/100 [00:00<?, ?it/s]

processing train_1


processing results:   1%|          | 1/100 [00:44<1:13:37, 44.62s/it]

processing train_17


processing results:   2%|▏         | 2/100 [01:22<1:06:02, 40.44s/it]

...

processing results:  98%|█████████▊| 98/100 [1:20:52<01:57, 58.76s/it]

processing test_644


processing results:  99%|█████████▉| 99/100 [1:21:22<00:50, 50.19s/it]

processing test_652


processing results: 100%|██████████| 100/100 [1:22:00<00:00, 49.21s/it]


In [9]:
with open(f'target_token_probs_train_{id}_10_epochs_prompt_2_with_shadow_with_world.json', 'w') as f:
    json.dump(all_model_probs, f)